In [15]:
import json
import cv2
import pandas as pd

In [16]:
results_file = './results.json'
labels_dir = './test/unit_labels/test/'
img_dir = './test/images/test/'

In [17]:
with open(results_file, 'r') as fp:
    results = json.load(fp)

In [18]:
for i, row in enumerate(results):
    for s in row['symbols']:
        s['used'] = 0

In [19]:
column_names = ["actual", "predicted", "correct", "incorrect", "missed"] #, "x1", "x2", "y1", "y2"]
df = pd.DataFrame(columns=column_names)
for i, row in enumerate(results):
    cur_img = row['img'].strip()
    img = cv2.imread(f"{img_dir}{cur_img}")
    img_txt = cur_img.split('.')[0] + '.txt'
    with open(f'{labels_dir}{img_txt}') as fp:
        for line in fp:
            splits = line.strip().split(" ")
            x_c, y_c, h, w = map(float, splits[1:])
            actual = splits[0].strip().split("__")[1:]
            height, width, channels = img.shape
            x_c, y_c, w, h = float(x_c)*width, float(y_c)*height, float(w)*width, float(h)*height
            
            predicted = []
            for s in row['symbols']:
                if s['used'] == 0:
                    if s['xmin'] <= x_c and x_c <= s['xmax'] and s['ymin'] <= y_c and y_c <= s['ymax']:
                        predicted = s['labels']
                        s['used'] = 1
                        break
            
            dif = set(actual) - set(predicted)
            missed = len(dif)
            correct = len(actual) - missed
            incorrect = len(predicted) - correct
                        
            df = pd.concat([df, pd.DataFrame([[actual, predicted, correct, incorrect, missed]], columns=column_names)], ignore_index=True)
for row in results:
    for s in row['symbols']:
        if s['used'] == 0:
            df = pd.concat([df, pd.DataFrame([[[], s['labels'], 0, 0, len(s['labels'])]], columns=column_names)], ignore_index=True)


In [20]:
df.head()

,actual,predicted,correct,incorrect,missed
0,"[mortar, company]","[mortar, company]",2,0,0
1,"[supply, armour, company]","[armour, company]",2,0,1
2,"[armour, half-platoon]",[company],0,1,2
3,"[infantry, company]",[company],1,0,1
4,"[supply, squad]",[company],0,1,2
